In [ ]:
import pandas as pd
import json

# Load data

In [ ]:
with open("../data/mg_als_up_gene_list.json", "r") as f:
    mg_als = json.load(f)
#with open("../data/mg_g29_up_gene_list.json", "r") as f:
#    mg_g29 = json.load(f)
#with open("../data/mg_c9_up_gene_list.json", "r") as f:
#    mg_c9 = json.load(f)

mn_expr = pd.read_csv("../data/mn_tpm.csv", index_col=0)

ccdb = pd.read_csv("../data/interaction_input_CellChatDB.csv", index_col=0)

# Extract ligand and receptors from CellChatDB

In [ ]:
def _split_tokens(s):
    """Split by '_' into tokens, handle NaN/empty, strip whitespace, drop empty."""
    if pd.isna(s) or s == "":
        return []
    return [t.strip() for t in str(s).split('_') if t.strip() != ""]


def extract_all_ligands(df, ligand_col="ligand"):
    """
    Unique set of tokens from the ligand column, splitting on '_'.
    Returns a sorted list.
    """
    tokens = (
        df[ligand_col]
        .astype(str)
        .apply(_split_tokens)
        .explode()
        .dropna()
    )
    return sorted(tokens.unique())


def extract_all_receptors(df, interaction_col="interaction_name", ligand_col="ligand"):
    """
    For each row:
      - split interaction_name and ligand by '_'
      - take tokens in interaction_name that are NOT in ligand
    Then union across all rows and return unique tokens (sorted list).
    """
    inter_tokens = df[interaction_col].apply(_split_tokens)
    lig_tokens = df[ligand_col].apply(_split_tokens)

    # row-wise set difference
    row_diffs = [
        set(i) - set(l)
        for i, l in zip(inter_tokens, lig_tokens)
    ]

    # merge to one unique set
    unique = set().union(*row_diffs) if row_diffs else set()
    return sorted(unique)

In [ ]:
all_ligands = extract_all_ligands(ccdb, ligand_col="ligand")
all_receptors = extract_all_receptors(ccdb, interaction_col="interaction_name", ligand_col="ligand")
print(f"Number of interactions in CellChatDB: {ccdb.shape[0]}")
print(f"Number of ligand genes in CellChatDB: {len(all_ligands)}")
print(f"Number of receptor genes in CellChatDB: {len(all_receptors)}")

# Filter for expressed receptors in MN

In [ ]:
tpm_thresholds = [0.5, 1, 2, 5, 7, 10]

for i in tpm_thresholds:
    mn_genes = mn_expr[mn_expr.gt(i).sum(axis=1) > (mn_expr.shape[1] * 0.5)].index.values
    
    print(f"At TPM >= {i}, number of expressed genes in MN detected: {len(mn_genes)} \
    {len(sorted(set(mn_genes) & set(all_receptors)))} of which are receptors in CellChatDB")

We decide the final threshold based on removal of known glial receptors while preserving the most receptors. In this case, it was 1.

In [ ]:
mn_genes = mn_expr[mn_expr.ge(1).sum(axis=1) > (mn_expr.shape[1] * 0.5)].index.values
mn_recs = sorted(set(mn_genes) & set(all_receptors))
print(f"Number of expressed genes in MN detected: {len(mn_genes)}")
print(f"Number of which are receptors in CellChatDB: {len(mn_recs)}")
print("TPM > 1 is chosen because it includes more genes but filtered out clear microglial signals like CX3CR1 and TREM2")

# Filter for ligands-of-interest in ALS MGs

In [ ]:
mg_als_ligs = sorted(set(mg_als) & set(all_ligands))
#mg_g29_ligs = sorted(set(mg_g29) & set(all_ligands))
#mg_c9_ligs = sorted(set(mg_c9) & set(all_ligands))

allowed = set(mn_recs) | set(mg_als_ligs)

mask = ccdb['interaction_name'].apply(
    lambda x: all(part in allowed for part in str(x).split('_'))
)

ccdb_filtered = ccdb[mask]

ccdb_secreted = ccdb_filtered.loc[ccdb_filtered.annotation == "Secreted Signaling", :]

ccdb_secreted.to_csv("../data/ccdb_als_mg_mn_lr_secreted.csv")